In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import time
import gc
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings 
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
app_train = pd.read_csv("../input/home-credit-default-risk/application_train.csv")
print("Training data shape: ", app_train.shape)
app_train.head()

In [ ]:
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()


data 살펴보기

In [ ]:
print('Training data shape: ', app_train.shape)
print('Testing data shape: ', app_test.shape)


In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train.shape

In [ ]:
app_train['TARGET'].value_counts()/app_train.shape[0]

TARGET 값에 대한 데이터 분포를 그래프로 확인

In [ ]:
app_train['TARGET'].astype(int).plot.hist()
#불균형 한 데이터 => 차이가 극심해 결과가 제대로 나오기 힘듦


target 값에 따른 amt_income_total(소득)값 분포 확인하기 위해 소득 데이터 확인 

In [ ]:
app_train['AMT_INCOME_TOTAL'].hist()
#분포가 한곳으로 몰려있어서 제대로 파악이 어려움 

다른 방식으로 확인

In [ ]:
sns.distplot(app_train['AMT_INCOME_TOTAL'])

In [ ]:
sns.boxplot(app_train['AMT_INCOME_TOTAL'])

일정 소득 이하인 데이터만 추출해 그래프로 표현

In [ ]:
condition_1 = app_train['AMT_INCOME_TOTAL'] < 1000000
app_train[app_train['AMT_INCOME_TOTAL'] < 1000000 ]

In [ ]:
app_train[app_train['AMT_INCOME_TOTAL'] < 1000000]['AMT_INCOME_TOTAL'].hist()
#20만원에 대부분 분포하고 있

In [ ]:
sns.distplot(app_train[condition_1]['AMT_INCOME_TOTAL'])

TARGET 값에 따른 AMT_INCOME_TOTAL값 분포도 비교

In [ ]:
cond1 = (app_train['TARGET'] == 1)
cond0 = (app_train['TARGET'] == 0)

cond_amt = (app_train['AMT_INCOME_TOTAL'] < 500000 )


In [ ]:
fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)

sns.violinplot(x='TARGET', y = 'AMT_INCOME_TOTAL', data = app_train[cond_amt], ax=ax[0][0])
sns.distplot(app_train[cond0 & cond_amt]['AMT_INCOME_TOTAL'], label='0', color='blue', ax = ax[0][1])
sns.distplot(app_train[cond1 & cond_amt]['AMT_INCOME_TOTAL'], label='1', color='red', ax = ax[0][1])

#별 차이가 없어 보이는데 TARGET 0이 좀 더 소득분포에서 낮은 곳에 밀집해있다.

app_train과 app_test를 한번에 합쳐 preprocessing수행

In [ ]:
app_train.shape, app_test.shape

In [ ]:
apps = pd.concat([app_train, app_test])

In [ ]:
apps.info()

In [ ]:
apps['TARGET'].value_counts(dropna=False)

In [ ]:
apps.dtypes.index

In [ ]:
object_columns = apps.dtypes[apps.dtypes=='object'].index.tolist()

In [ ]:
apps['CODE_GENDER'] = pd.factorize(apps['CODE_GENDER'])[0]

In [ ]:
for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

Null 값 일괄 변환

In [ ]:
apps.isnull().sum().head(100)

In [ ]:
apps = apps.fillna(-999)

In [ ]:
apps.isnull().sum().head(100)

학습 데이터와 테스트 데이터 다시 분리

In [ ]:
app_train2 = apps[apps['TARGET'] != -999]
app_test2 = apps[apps['TARGET'] == -999]
app_train2.shape, app_test2.shape

In [ ]:
app_test2 = app_test2.drop('TARGET', axis = 1, inplace=False)
app_test2.shape

학습 데이터를 검증 데이터로 분리하고 LGBM Classfier로 학습 수행 

In [ ]:
ftr_app = app_train2.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train2['TARGET']

In [ ]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state = 2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
    n_jobs = -1,
    n_estimators = 1000, 
    learning_rate = 0.02,
    num_leaves = 32,
    subsample=0.8,
    max_depth=12,
    silent = -1,
    verbose =-1
)


In [ ]:
clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=100, early_stopping_rounds = 50)

feature 중요도 시각화

In [ ]:
from lightgbm import plot_importance
plot_importance(clf, figsize=(16, 32))

feature importance에서 중요하게 나왔던 칼럼을 가지고 target값이 각각 0, 1일때 histogram시각화

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')

In [ ]:
def show_hist_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=ax[0][0])
        sns.distplot(df[cond_1][column], label='1', color='red', ax=ax[0][1])
        sns.distplot(df[cond_0][column], label='0', color='blue', ax=ax[0][1])
        
        

In [ ]:
columns = ['AMT_INCOME_TOTAL','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
           'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 
           'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']


In [ ]:
show_hist_by_target(app_train, columns)
#나이가 많을수록 대출 상환이 잘 되고, 적을 수록 대출 상환하지 않는 사람이 많다.
#일한 기간이 적을수록 대출을 연체한다.
#ext_source 1, 2, 3에서 target이 0과 1일때 그래프 차이가 크다 
#신용평가가 낮은 사람들일수록 대출금 상환이 이루어지지 않는다.

차이가 많았던 feature - DAYS_BIRTH 분석

In [ ]:
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

고객 나이에 대한 히스토그램 분포 확인

In [ ]:
plt.style.use('fivethirtyeight')

plt.hist(app_train['DAYS_BIRTH']/365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');

#나이는 20~70(청년부터 노년)까지 고르게 분포해있음

나이가 TARGET값에 어떤 영향이 있는지 확인

In [ ]:
plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages'); 

#젊을수록 대출을 제때 상환하지 못한다고 판단할 수 있다.

차이가 많았던 feature -  EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3 분석

상관관계가 가장 높았던 EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3 시각화

In [ ]:
# TARGET 변수와 EXT_SOURCE와의 상관관계와 EXT_SOURCE 서로간의 상관관계를 살핌

ext_data=app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
ext_data_corrs=ext_data.corr()
ext_data_corrs


In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

In [ ]:
# 각 EXT_SOURCE 를 TARGET값 별로 나눠서 분포를 살핌
plt.figure(figsize=(10,12))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,source],label='target==0', color = 'green')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,source],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');
plt.tight_layout(h_pad=2.5)

#ext_source 1, 2, 3에서 신용도가 높을수록 대출 상환이 잘 이루어지는 것이 나왔고, 그 차이가 3에서 가장 크기 때문에 가장 영향력 있는 feature가 될 것으로 예상됨

object 컬럼)을 TARGET 유형에 따라 Count 비교

In [ ]:
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_columns

In [ ]:
def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=65)

show_category_by_target(app_train, object_columns)

성별과 대출 연체의 상관관계에 대해 분석

In [ ]:
# 대출 횟수 대비 연체 비율이 남성이 여성보다 높음 --> 이를 value_counts()로 확인

cond_1 = (app_train['TARGET'] == 1)
cond_0 = (app_train['TARGET'] == 0)
print(app_train['CODE_GENDER'].value_counts()/app_train.shape[0])

#여성 대출 비율이 높음



In [ ]:
print('\n연체인 경우\n',app_train[cond_1]['CODE_GENDER'].value_counts()/app_train[cond_1].shape[0])
# 숫자만 봐선 여성 연체가 높아보이지만 대출 비율과 함께 봤을때 남성 연체 비율이 훨씬 높음을 알 수 있다.

In [ ]:
print('\n연체가 아닌 경우\n',app_train[cond_0]['CODE_GENDER'].value_counts()/app_train[cond_0].shape[0])

데이터 가공 전 학습과 테스트용 데이터 세트 결합

In [ ]:
apps = pd.concat([app_train, app_test])
print(apps.shape)

Application 데이터 세트에 다양한 feature engineering 수행

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].head()

In [ ]:
#컬럼 방향으로 평균을 구하기 위해서 axis=1 추가
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

In [ ]:
# mean과 std값 구함
apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_STD']].head(10)

In [ ]:
#nan값 제거
apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_STD']].head(10)

In [ ]:
# AMT_CREDIT 비율로 Feature 가공

apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']

In [ ]:
# AMT_INCOME_TOTAL 비율로 Feature 가공

# AMT_INCOME_TOTAL 비율로 대출 금액 관련 피처 가공
apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
# 가족수를 고려한 가처분 소득 피처 가공. 
apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']

In [ ]:
# DAYS_BIRTH, DAYS_EMPLOYED 비율로 Feature 가공

# DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공. 
apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']

In [ ]:
# 데이터 레이블 인코딩

object_columns = apps.dtypes[apps.dtypes=='object'].index.tolist()

for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

In [ ]:
apps_train = apps[-apps['TARGET'].isnull()]
apps_test = apps[apps['TARGET'].isnull()]
apps.shape, apps_train.shape, apps_test.shape

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))